In [6]:
import keras

from keras import Model
from keras.layers import Conv2D, MaxPool2D, BatchNormalization, Input
from keras.layers import SeparableConv2D, MaxPool1D,AveragePooling2D,AveragePooling1D,GlobalAveragePooling2D
from keras.layers import CuDNNGRU as GRU
from keras.layers import CuDNNLSTM as LSTM
from keras.layers import Reshape, Bidirectional, Flatten, Dense, Permute, Multiply, Average
from keras.preprocessing import image

import tqdm
import os
import time
import numpy as np
import json


data_root = '/home/philip/data/Keyword_spot/'

In [3]:
batch_size = 64

training_data_gen = image.ImageDataGenerator(width_shift_range=0.1)
training_gen = training_data_gen.flow_from_directory(data_root + "train/original_all", class_mode="categorical",
                                                     target_size=(128, 63),
                                                     batch_size=batch_size,
                                                     color_mode="grayscale")

Found 68606 images belonging to 31 classes.


In [ ]:
import tensorflow as tf
sess = tf.InteractiveSession()

from keras import backend as K
K.set_session(sess)

In [15]:
conv_size = (7, 7)
conv_filter = [96, 192, 256, 256]
dilation = [4, 4, 4, 4]
RNN_Layer = [GRU, LSTM][1]
RNN_repeat = 0
class_num = training_gen.num_classes
attention = False
incept_conv_size = [(1,1),(7,7),(3,5),(5,3)]

def incept_block(inputs,filters):
    a = [SeparableConv2D(filters,size,padding='same',activation='relu')(inputs) for size in incept_conv_size]
    b = Average()(a)
    return b
def SE_block(inputs,filters):
    x = GlobalAveragePooling2D()(inputs)
    x = Dense(filters//8,activation='relu')(x)
    x = Dense(filters,activation='sigmoid')(x)
    x = Multiply()([inputs,x])
    return x

def get_model():
    inputs = Input((128, 63, 1))
    # x = Conv2D(conv_filter[0], conv_size,
    #            padding='same', dilation_rate=dilation[0])(inputs)
    x = incept_block(inputs,conv_filter[0])
    x = SE_block(x,conv_filter[0])
    x = BatchNormalization()(x)
    x = MaxPool2D((4, 1))(x)
    # x = Conv2D(conv_filter[1], conv_size, padding='same',
    #            dilation_rate=dilation[1])(x)
    x = incept_block(x,conv_filter[1])
    x = SE_block(x,conv_filter[1])
    x = BatchNormalization()(x)
    x = MaxPool2D((4, 1))(x)

    # x = Conv2D(conv_filter[2], conv_size, padding='same',
    #            dilation_rate=dilation[2])(x)
    x = incept_block(x,conv_filter[2])
    x = SE_block(x,conv_filter[2])

    x = BatchNormalization()(x)
    x = MaxPool2D((4, 1))(x)

    x = Conv2D(conv_filter[3], conv_size, padding='same',
               dilation_rate=dilation[3])(x)
#     x = incept_block(x,conv_filter[3])
#     x = SE_block(x,conv_filter[3])

    x = BatchNormalization()(x)
    x = MaxPool2D((2, 1))(x)
    x = Reshape((63, conv_filter[-1]))(x)
    for i in range(RNN_repeat):
        x = Bidirectional(
            RNN_Layer(conv_filter[-1], return_sequences=True), merge_mode='ave')(x)

    if attention:
        a = Permute((2, 1))(x)
        a = Dense(63, activation='softmax', input_dim=(63,))(a)
        a = Permute((2, 1))(a)
        x = Multiply()([x, a])

    o = MaxPool1D(63)(x)
    o = Flatten()(o)
    output = Dense(class_num, activation='softmax')(o)
    return Model(inputs, output)

In [16]:
model = get_model()

In [17]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 128, 63, 1)   0                                            
__________________________________________________________________________________________________
separable_conv2d_29 (SeparableC (None, 128, 63, 96)  193         input_5[0][0]                    
__________________________________________________________________________________________________
separable_conv2d_30 (SeparableC (None, 128, 63, 96)  241         input_5[0][0]                    
__________________________________________________________________________________________________
separable_conv2d_31 (SeparableC (None, 128, 63, 96)  207         input_5[0][0]                    
__________________________________________________________________________________________________
separable_

In [ ]:
model = keras.applications.MobileNet(include_top=True,weights=None,input_shape=(128,63,1),classes=31,pooling='ave')

In [ ]:
from nasnet import NASNetMobile
model = NASNetMobile(include_top=True,weights=None,input_shape=(128,63,1),classes=31,pooling='max')

In [ ]:
# model.summary()

In [ ]:
sgd = keras.optimizers.SGD(lr=0.01, momentum=0.9)
adadelta = keras.optimizers.Adadelta(
    lr=1, rho=0.95, epsilon=1e-08, decay=0.0)
adam = keras.optimizers.Adam()
optimizer = [sgd, adadelta, adam][0]

model.compile(optimizer=optimizer, loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])


In [ ]:
# now = time.localtime(time.time())
# time_folder = "%d_%d_%d_%d_%d" % (
#     now.tm_mon, now.tm_mday, now.tm_hour, now.tm_min, now.tm_sec)
# print(time_folder)

model_name = "NAS"
model_folder = os.path.join(
    "/home/philip/data/Keyword_spot/saved_model/", model_name)
if not os.path.isdir(model_folder):
    os.mkdir(model_folder)

In [ ]:
with open(os.path.join(model_folder,'SUMMARY.txt'),'w') as f:
    f.write(str(training_gen.class_indices))

In [ ]:
json_config = model.to_json()
with open(os.path.join(model_folder,'config.json'),'w') as json_file:
    json.dump(json_config,json_file)

In [ ]:
with open(os.path.join(model_folder,'config.json'),'r') as json_file:
    recover = json.load(json_file)
model_recover = keras.models.model_from_json(recover)
model_recover.load_weights(os.path.join(model_folder,'weight_3.h5'))

In [ ]:
model_recover.compile(optimizer='sgd', loss=keras.losses.categorical_crossentropy,
              metrics=[keras.metrics.categorical_accuracy])

In [ ]:
# model.compile(optimizer='adam', loss=keras.losses.categorical_crossentropy,
#               metrics=[keras.metrics.categorical_accuracy])
# model.load_weights(os.path.join(model_folder,'weight_1.h5'))

history = model_recover.fit_generator(training_gen, steps_per_epoch=training_gen.n // training_gen.batch_size + 1, epochs=3)

In [ ]:
for i in range(1,6):
    print("circle %d" % (i))
    history = model_recover.fit_generator(training_gen, steps_per_epoch=training_gen.n // training_gen.batch_size + 1, epochs=10)
    model_recover.save_weights(os.path.join(model_folder+"/weight_%d.h5")%(i+1))

In [ ]:
for i in range(5):
    print("circle %d" % (i))
    history = model.fit_generator(training_gen, steps_per_epoch=training_gen.n // training_gen.batch_size + 1, epochs=5)
    model.save(os.path.join(model_folder,"NASmobile_%d.h5"%(i+1)))

In [ ]:
model.save(os.path.join(model_folder, "%s_%d.h5" % (model_name,(i + 1))))

In [ ]:
model = keras.models.load_model('/home/philip/data/Keyword_spot/saved_model/Incept_like_attention_no_RNN/Incept_like_attention_no_RNN_5.h5')

In [ ]:
import predict_test
predict_test.predict_test(model,training_gen.class_indices,0.2)

In [ ]:
model.save(os.path.join(model_folder, "%s.h5" % (model_name)))

In [ ]:
import predict_test
import importlib
importlib.reload(predict_test)
predict_test.predict_test(model_recover,training_gen.class_indices,0.3)